# Chapter 13: Sampling Plans

In [2]:
import numpy as np

## Algorithm 13.1

In [9]:
def samples_full_factorial(a, b, m):
    ranges = (np.linspace(a[i], b[i], m[i] ) for i in range(len(a)))
    return np.meshgrid(*ranges)

## Algorithm 13.2

In [19]:
def uniform_projection_plan(m, n):
    perms = [ np.random.permutation(m) for i in range(n) ]
    return [ [perms[i][j] for i in range(n)] for j in range(m) ]